# Mạng tạo sinh

Mạng Nơ-ron Hồi quy (RNNs) và các biến thể tế bào có cổng của chúng như Tế bào Bộ nhớ Ngắn Dài (LSTMs) và Đơn vị Hồi quy Có cổng (GRUs) cung cấp một cơ chế để mô hình hóa ngôn ngữ, tức là chúng có thể học cách sắp xếp từ và đưa ra dự đoán cho từ tiếp theo trong một chuỗi. Điều này cho phép chúng ta sử dụng RNNs cho các **nhiệm vụ tạo sinh**, chẳng hạn như tạo văn bản thông thường, dịch máy, và thậm chí là chú thích hình ảnh.

Trong kiến trúc RNN mà chúng ta đã thảo luận ở đơn vị trước, mỗi đơn vị RNN tạo ra trạng thái ẩn tiếp theo như một đầu ra. Tuy nhiên, chúng ta cũng có thể thêm một đầu ra khác vào mỗi đơn vị hồi quy, điều này cho phép chúng ta xuất ra một **chuỗi** (có độ dài bằng với chuỗi ban đầu). Hơn nữa, chúng ta có thể sử dụng các đơn vị RNN không nhận đầu vào ở mỗi bước, mà chỉ lấy một vector trạng thái ban đầu, sau đó tạo ra một chuỗi các đầu ra.

Trong notebook này, chúng ta sẽ tập trung vào các mô hình tạo sinh đơn giản giúp chúng ta tạo văn bản. Để đơn giản, hãy xây dựng **mạng cấp độ ký tự**, mạng này tạo văn bản từng chữ cái một. Trong quá trình huấn luyện, chúng ta cần lấy một tập hợp văn bản và chia nó thành các chuỗi ký tự.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Xây dựng từ vựng ký tự

Để xây dựng mạng sinh cấp độ ký tự, chúng ta cần chia văn bản thành các ký tự riêng lẻ thay vì các từ. Điều này có thể thực hiện bằng cách định nghĩa một bộ tách từ khác:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Hãy xem ví dụ về cách chúng ta có thể mã hóa văn bản từ tập dữ liệu của mình:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Huấn luyện RNN tạo sinh

Cách chúng ta sẽ huấn luyện RNN để tạo văn bản như sau. Ở mỗi bước, chúng ta sẽ lấy một chuỗi ký tự có độ dài `nchars`, và yêu cầu mạng tạo ra ký tự đầu ra tiếp theo cho mỗi ký tự đầu vào:

![Hình minh họa ví dụ RNN tạo ra từ 'HELLO'.](../../../../../translated_images/vi/rnn-generate.56c54afb52f9781d.webp)

Tùy thuộc vào tình huống thực tế, chúng ta cũng có thể muốn bao gồm một số ký tự đặc biệt, chẳng hạn như *kết thúc chuỗi* `<eos>`. Trong trường hợp của chúng ta, mục tiêu là huấn luyện mạng để tạo văn bản liên tục, vì vậy chúng ta sẽ cố định kích thước của mỗi chuỗi bằng số lượng token `nchars`. Do đó, mỗi ví dụ huấn luyện sẽ bao gồm `nchars` đầu vào và `nchars` đầu ra (là chuỗi đầu vào được dịch sang trái một ký tự). Minibatch sẽ bao gồm một số chuỗi như vậy.

Cách chúng ta tạo minibatch là lấy mỗi văn bản tin tức có độ dài `l`, và tạo tất cả các kết hợp đầu vào-đầu ra có thể từ nó (sẽ có `l-nchars` kết hợp như vậy). Những kết hợp này sẽ tạo thành một minibatch, và kích thước của các minibatch sẽ khác nhau ở mỗi bước huấn luyện.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Bây giờ chúng ta sẽ định nghĩa mạng tạo. Nó có thể dựa trên bất kỳ tế bào hồi quy nào mà chúng ta đã thảo luận trong đơn vị trước (đơn giản, LSTM hoặc GRU). Trong ví dụ của chúng ta, chúng ta sẽ sử dụng LSTM.

Vì mạng nhận các ký tự làm đầu vào và kích thước từ vựng khá nhỏ, chúng ta không cần lớp nhúng, đầu vào được mã hóa một-hot có thể trực tiếp đi vào tế bào LSTM. Tuy nhiên, vì chúng ta truyền số ký tự làm đầu vào, chúng ta cần mã hóa một-hot chúng trước khi truyền vào LSTM. Điều này được thực hiện bằng cách gọi hàm `one_hot` trong quá trình `forward`. Bộ mã hóa đầu ra sẽ là một lớp tuyến tính, lớp này sẽ chuyển trạng thái ẩn thành đầu ra được mã hóa một-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Trong quá trình huấn luyện, chúng ta muốn có khả năng lấy mẫu văn bản được tạo ra. Để làm điều đó, chúng ta sẽ định nghĩa hàm `generate`, hàm này sẽ tạo ra chuỗi đầu ra có độ dài `size`, bắt đầu từ chuỗi ban đầu `start`.

Cách hoạt động như sau. Đầu tiên, chúng ta sẽ truyền toàn bộ chuỗi bắt đầu qua mạng, và lấy trạng thái đầu ra `s` cùng với ký tự dự đoán tiếp theo `out`. Vì `out` được mã hóa dưới dạng one-hot, chúng ta sử dụng `argmax` để lấy chỉ số của ký tự `nc` trong từ vựng, và dùng `itos` để xác định ký tự thực tế rồi thêm nó vào danh sách các ký tự kết quả `chars`. Quá trình tạo một ký tự này được lặp lại `size` lần để tạo ra số lượng ký tự yêu cầu.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Bây giờ hãy bắt đầu huấn luyện! Vòng lặp huấn luyện gần như giống hệt với tất cả các ví dụ trước đây của chúng ta, nhưng thay vì in độ chính xác, chúng ta sẽ in văn bản được tạo mẫu sau mỗi 1000 epoch.

Cần chú ý đặc biệt đến cách chúng ta tính toán loss. Chúng ta cần tính loss dựa trên đầu ra được mã hóa one-hot `out` và văn bản mong đợi `text_out`, là danh sách các chỉ số ký tự. May mắn thay, hàm `cross_entropy` yêu cầu đầu ra mạng chưa được chuẩn hóa làm đối số đầu tiên và số lớp làm đối số thứ hai, điều này hoàn toàn phù hợp với những gì chúng ta có. Nó cũng tự động thực hiện việc tính trung bình trên kích thước minibatch.

Chúng ta cũng giới hạn việc huấn luyện bởi số mẫu `samples_to_train`, để không phải chờ quá lâu. Chúng tôi khuyến khích bạn thử nghiệm và thử huấn luyện lâu hơn, có thể trong vài epoch (trong trường hợp đó, bạn sẽ cần tạo một vòng lặp khác xung quanh đoạn mã này).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Ví dụ này đã tạo ra một văn bản khá tốt, nhưng vẫn có thể cải thiện thêm theo một số cách:

* **Cải thiện việc tạo minibatch**. Cách chúng ta chuẩn bị dữ liệu để huấn luyện là tạo một minibatch từ một mẫu duy nhất. Điều này không lý tưởng, vì các minibatch có kích thước khác nhau, và một số thậm chí không thể được tạo ra vì văn bản nhỏ hơn `nchars`. Ngoài ra, các minibatch nhỏ không tận dụng GPU đủ hiệu quả. Sẽ hợp lý hơn nếu lấy một đoạn văn bản lớn từ tất cả các mẫu, sau đó tạo tất cả các cặp đầu vào-đầu ra, xáo trộn chúng, và tạo các minibatch có kích thước bằng nhau.

* **LSTM nhiều lớp**. Có lý do để thử 2 hoặc 3 lớp tế bào LSTM. Như đã đề cập trong phần trước, mỗi lớp của LSTM trích xuất các mẫu nhất định từ văn bản, và trong trường hợp trình tạo cấp độ ký tự, chúng ta có thể kỳ vọng lớp LSTM thấp hơn chịu trách nhiệm trích xuất âm tiết, và các lớp cao hơn - cho từ và các tổ hợp từ. Điều này có thể được thực hiện đơn giản bằng cách truyền tham số số lượng lớp vào hàm khởi tạo LSTM.

* Bạn cũng có thể muốn thử nghiệm với **các đơn vị GRU** để xem loại nào hoạt động tốt hơn, và với **các kích thước lớp ẩn khác nhau**. Lớp ẩn quá lớn có thể dẫn đến overfitting (ví dụ: mạng sẽ học chính xác văn bản), và kích thước nhỏ hơn có thể không tạo ra kết quả tốt.


## Tạo văn bản mềm và nhiệt độ

Trong định nghĩa trước của `generate`, chúng ta luôn chọn ký tự có xác suất cao nhất làm ký tự tiếp theo trong văn bản được tạo. Điều này dẫn đến việc văn bản thường "lặp lại" giữa các chuỗi ký tự giống nhau nhiều lần, như trong ví dụ sau:
```
today of the second the company and a second the company ...
```

Tuy nhiên, nếu chúng ta xem xét phân phối xác suất cho ký tự tiếp theo, có thể thấy rằng sự khác biệt giữa một vài xác suất cao nhất không lớn, ví dụ: một ký tự có xác suất là 0.2, ký tự khác là 0.19, v.v. Chẳng hạn, khi tìm ký tự tiếp theo trong chuỗi '*play*', ký tự tiếp theo có thể là khoảng trắng hoặc **e** (như trong từ *player*).

Điều này dẫn chúng ta đến kết luận rằng không phải lúc nào cũng "công bằng" khi chọn ký tự có xác suất cao hơn, vì việc chọn ký tự có xác suất cao thứ hai vẫn có thể dẫn đến văn bản có ý nghĩa. Sẽ hợp lý hơn nếu **lấy mẫu** ký tự từ phân phối xác suất được cung cấp bởi đầu ra của mạng.

Việc lấy mẫu này có thể được thực hiện bằng hàm `multinomial`, hàm này triển khai cái gọi là **phân phối đa thức**. Một hàm thực hiện việc tạo văn bản **mềm** này được định nghĩa dưới đây:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Chúng tôi đã giới thiệu một tham số mới gọi là **nhiệt độ**, được sử dụng để chỉ ra mức độ chúng ta nên tuân thủ xác suất cao nhất. Nếu nhiệt độ là 1.0, chúng ta thực hiện lấy mẫu đa thức công bằng, và khi nhiệt độ tăng lên vô hạn - tất cả các xác suất trở nên bằng nhau, và chúng ta chọn ngẫu nhiên ký tự tiếp theo. Trong ví dụ dưới đây, chúng ta có thể quan sát rằng văn bản trở nên vô nghĩa khi chúng ta tăng nhiệt độ quá nhiều, và nó giống như văn bản "lặp lại" được tạo cứng khi nhiệt độ tiến gần đến 0.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
